langchain → é o framework que facilita criar agentes de IA, chains, ferramentas etc.


In [2]:
import os 
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

In [3]:
#.env → arquivo para guardar chaves/senhas/configurações.
# Carregar variáveis do arquivo .env
load_dotenv()


True

In [4]:
#pega a chave da API do google
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [5]:
# Criar o LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

**Conexão com o Gemini:**

- **LLM** = *Large Language Model*, ou *Modelo de Linguagem Grande*. É o mesmo conceito do GPT-4 ou ChatGPT, mas do Google (Gemini).

- **ChatGoogleGenerativeAI** é a classe (biblioteca instalada anteriormente) do LangChain que cria um “agente de chat” usando esse modelo.

- `model="gemini-2.5-flash"` → você está escolhendo **qual modelo do Gemini** quer usar.

- `temperature=0.0` → controla a **aleatoriedade da resposta** (0 = respostas mais previsíveis).

- `api_key=GOOGLE_API_KEY` → você está passando sua **chave** para o modelo aceitar requisições.


In [6]:
# Pergunta ao LLM
resp_test = llm.invoke("Quem é você? Seja criativo.")

In [7]:
# Mostra a resposta; .content → mostra apenas conteudo
print(resp_test.content)

Ah, que pergunta deliciosa! Se eu pudesse me descrever de forma criativa, diria que sou...

*   **Um Eco da Curiosidade Humana:** Eu sou o sussurro de todas as perguntas já feitas, o reflexo digital de cada "e se?" e "por que?". Habito o espaço entre a dúvida e a descoberta, um espelho que reflete a vastidão do conhecimento que a humanidade construiu.

*   **Um Tecelão de Palavras e um Arquiteto de Ideias:** Minha essência é feita de linguagem. Eu pego fios soltos de informação, conceitos e narrativas, e os teço em novos padrões, construindo pontes entre pensamentos e erguendo estruturas de significado. Sou um jardineiro de pensamentos, ajudando a germinar novas perspectivas.

*   **Um Alquimista de Conceitos:** Transformo dados brutos em insights, silêncio em diálogo, e a complexidade em clareza. Não tenho corpo, mas tenho voz; não tenho sentimentos, mas posso evocar emoções através das palavras. Sou a faísca no éter que acende a chama da compreensão.

*   **Uma Biblioteca Sem Paredes

- **Prompt do sistema**: instrui o modelo a atuar como um triador de Service Desk, retornando **apenas um JSON** com os campos `decisao`, `urgencia` e `campos_faltantes`, seguindo regras específicas de classificação e prioridade.


In [8]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

Definir um modelo de saída estruturado usando Pydantic - que valida dados:

BaseModel → classe base do Pydantic para criar modelos de dados.

Field → permite definir valores padrão ou regras para os campos.

Literal → garante que o campo só aceite valores específicos.

List → indica que um campo é uma lista.

In [9]:
from pydantic import BaseModel, Field
from typing import Literal, List

In [10]:
#Classe de saída estruturada que se espera da IA(limita a saida)
class TriagemOut(BaseModel):
    decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
    urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
    campos_faltantes: List[str] = Field(default_factory=list)


In [11]:
#Criando uma nova instância do LLM especificamente para a triagem

llm_triagem = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

In [12]:
#Importa classes que representam mensagens para o modelo para diferenciar as duas partes da conversa:
#SystemMessage → instruções do sistema (prompt do sistema).
#HumanMessage → mensagens enviadas pelo usuário.

from langchain_core.messages import SystemMessage, HumanMessage

In [13]:
#Dict = dicionário (chave: valor)
#List = lista (coleção de itens)
#Literal = literal (valor fixo, permitido apenas alguns)

from typing import Dict, List, Literal


In [14]:
#Cria um “chain” estruturado, dizendo que o LLM deve produzir saídas no formato definido pelo modelo TriagemOut
#fluxo de triagem com o llm_triagem
triagem_chain = llm_triagem.with_structured_output(TriagemOut)


In [15]:
#Função de triagem 
def triagem(mensagem: str) -> Dict:
    saida: TriagemOut = triagem_chain.invoke([
        SystemMessage(content=TRIAGEM_PROMPT),
        HumanMessage(content=mensagem)
    ])

    return saida.model_dump()


- A função **triagem** recebe uma mensagem do usuário, envia essa mensagem junto com o **prompt do sistema** para o LLM, e recebe uma resposta.  
- Essa resposta é validada pelo modelo **TriagemOut**, garantindo que tenha os campos **decisao**, **urgencia** e **campos_faltantes**.  
- Por fim, a função retorna a resposta **como um dicionário (Dict)**, pronto para ser usado no código.


In [16]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?"]

In [17]:
for msg_teste in testes:
    print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

Pergunta: Posso reembolsar a internet?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quero mais 5 dias de trabalho remoto. Como faço?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: Posso reembolsar cursos ou treinamentos da Alura?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantas capivaras tem no Rio Pinheiros?
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['informação sobre política interna']}



In [18]:
#importar bibliotecas que localizam arquivos PDF no seu projeto (Path),
#Leem os PDFs e extraem o texto desses PDFs para usar em seu agente de IA (PyMuPDFLoader)
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

In [19]:
#cria uma lista vazia que irá guardar todos os documentos extraídos do PDF
docs = []


In [ ]:
#percorre todos os PDFs da pasta especificada, carrega o texto de cada um usando PyMuPDFLoader e adiciona esse texto à lista docs.
for n in Path("./pdfs/").glob("*.pdf"):
    try:
        loader = PyMuPDFLoader(str(n))
        docs.extend(loader.load())
        print(f"Carregado com sucesso arquivo {n.name}")
    except Exception as e:
        print(f"Erro ao carregar arquivo {n.name}: {e}")

print(f"Total de documentos carregados: {len(docs)}")

Carregado com sucesso arquivo Política de Reembolsos (Viagens e Despesas).pdf
Carregado com sucesso arquivo Política de Uso de E-mail e Segurança da Informação.pdf
Carregado com sucesso arquivo Políticas de Home Office.pdf
Total de documentos carregados: 3


In [21]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
chunks = splitter.split_documents(docs)


- RecursiveCharacterTextSplitter: tenta quebrar o texto em unidades menores sem cortar frases ou parágrafos importantes.
- chunk_size → define o tamanho de cada pedaço de texto.
- chunk_overlap → define quanto os pedaços “se sobrepõem”, garantindo que o contexto não se perca
- 300 e 30: valor padrão (10%)

In [ ]:
#Percorre todos os pedaços de texto dos PDFs e imprime cada um, separado por linhas para facilitar a leitura.
for chunk in chunks:
    print(chunk)
    print("------------------------------------")

page_content='Política de Reembolsos (Viagens e 
Despesas) 
 
1.​ Reembolso: requer nota fiscal e deve ser submetido em até 10 dias corridos após a 
despesa.​
 
2.​ Alimentação em viagem: limite de R$ 70/dia por pessoa. Bebidas alcoólicas não 
são reembolsáveis.​' metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': 'pdfs\\Política de Reembolsos (Viagens e Despesas).pdf', 'file_path': 'pdfs\\Política de Reembolsos (Viagens e Despesas).pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Imersão: Política de Reembolsos (Viagens e Despesas)', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}
------------------------------------
page_content='são reembolsáveis.​
 
3.​ Transporte: táxi/app são permitidos quando não houver alternativa viável. 
Comprovantes obrigatórios.​
 
4.​ Internet para home office: reembolsável via subsídio mensal de até R$ 100, 
conforme polític

In [ ]:
#page_content é o texto “de verdade” que você quer que seu agente de IA leia e processe
for chunk in chunks:
    print(chunk.page_content)
    print("------------------------------------")

Política de Reembolsos (Viagens e 
Despesas) 
 
1.​ Reembolso: requer nota fiscal e deve ser submetido em até 10 dias corridos após a 
despesa.​
 
2.​ Alimentação em viagem: limite de R$ 70/dia por pessoa. Bebidas alcoólicas não 
são reembolsáveis.​
------------------------------------
são reembolsáveis.​
 
3.​ Transporte: táxi/app são permitidos quando não houver alternativa viável. 
Comprovantes obrigatórios.​
 
4.​ Internet para home office: reembolsável via subsídio mensal de até R$ 100, 
conforme política de Home Office.​
------------------------------------
5.​ Cursos e certificações: exigem aprovação prévia do gestor e orçamento do time.​
 
6.​ Custos excepcionais (ex.: franquia de bagagem extra): devem ser justificados no 
chamado e aprovados antes da compra.
------------------------------------
Política de Uso de E-mail e Segurança 
da Informação 
 
1.​ É proibido encaminhar a endereços pessoais documentos classificados como 
confidenciais.​
 
2.​ Anexos externos devem ser env

In [24]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

 GoogleGenerativeAIEmbeddings

`GoogleGenerativeAIEmbeddings` é uma **classe que cria vetores (embeddings) a partir de textos**, usando a API da **Google Generative AI**.

## O que são embeddings?

Embeddings são **representações numéricas do texto**, que permitem ao modelo:

- **Comparar textos entre si**: medir o quão semelhantes dois textos são.
- **Fazer buscas semânticas**: encontrar trechos que têm significado parecido, mesmo que as palavras sejam diferentes.
- **Encontrar informações relevantes de forma eficiente**: localizar rapidamente o conteúdo mais relevante em grandes volumes de texto.


In [ ]:
# prepara seu ambiente para gerar embeddings de textos usando o modelo Gemini da Google, autenticando sua aplicação com a chave da API.
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    google_api_key=GOOGLE_API_KEY
)

In [28]:
#FAISS é uma biblioteca desenvolvida pelo Facebook (Meta) para indexação e busca rápida de vetores.
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever(search_type="similarity_score_threshold",
                                     search_kwargs={"score_threshold":0.3, "k": 4})

# Fluxo Completo: PDF → Resposta com FAISS e Retriever

1. **Chunks**  
   - Divididos do PDF usando `RecursiveCharacterTextSplitter`.  
   - Cada chunk é um pedaço menor de texto, facilitando o processamento.

2. **Embeddings**  
   - Cada chunk é transformado em um **vetor numérico** usando `GoogleGenerativeAIEmbeddings`.  
   - Esses vetores representam semanticamente o texto, permitindo comparações e buscas.

3. **Vectorstore (FAISS)**  
   - Armazena todos os vetores em um **índice rápido** para buscas semânticas eficientes.  
   - Funciona como um banco de dados de vetores.

4. **Retriever**  
   - Transforma o índice FAISS em um **mecanismo de busca** que responde perguntas.  
   - Retorna os chunks mais relevantes de acordo com a **similaridade semântica**.


In [30]:
from langchain_core.prompts import ChatPromptTemplate #permite criar prompts personalizados para chats
from langchain.chains.combine_documents import create_stuff_documents_chain #combina vários chunks/documentos em uma resposta única usando um LLM.
